# Notebook 4: Knowledge Base Assistant

**User Stories**: US4 (Natural-Language CRM Query) + US3 (KB Search)  
**Persona**: Customer Service Representative (CSR)  

Ask natural-language questions against the Salesforce Knowledge Base
and get accurate answers with article citations.

In [ ]:
# Cell 2: Environment + Auth Setup
import os
from pathlib import Path

from dotenv import load_dotenv

env_path = Path("../.env")
if env_path.exists():
    load_dotenv(env_path)

required_vars = [
    "AZURE_AI_PROJECT_ENDPOINT",
    "AZURE_OPENAI_DEPLOYMENT",
    "SF_INSTANCE_URL",
    "SF_ACCESS_TOKEN",
]
missing = [v for v in required_vars if not os.environ.get(v)]
if missing:
    raise OSError(f"Missing required environment variables: {missing}")

print("Environment configured successfully.")
print(f"  Azure AI endpoint: {os.environ['AZURE_AI_PROJECT_ENDPOINT'][:40]}...")
print(f"  Salesforce instance: {os.environ['SF_INSTANCE_URL']}")

In [ ]:
# Cell 3: MCP Connection — salesforce-knowledge
import sys

from azure.ai.projects import AIProjectClient
from azure.ai.projects.models import McpToolConnection
from azure.identity import DefaultAzureCredential

credential = DefaultAzureCredential()
project_client = AIProjectClient(
    endpoint=os.environ["AZURE_AI_PROJECT_ENDPOINT"],
    credential=credential,
)

knowledge_mcp = McpToolConnection(
    server_label="salesforce-knowledge",
    command=f"{sys.executable} -m mcp_servers.salesforce_knowledge.server",
    env={
        "SF_INSTANCE_URL": os.environ["SF_INSTANCE_URL"],
        "SF_ACCESS_TOKEN": os.environ["SF_ACCESS_TOKEN"],
        "MCP_TRANSPORT": "stdio",
    },
)

print(f"MCP connection configured: {knowledge_mcp.server_label}")

In [ ]:
# Cell 4: Create Service Agent
from pathlib import Path as _Path

system_prompt_path = _Path("../agents/service/system_prompt.md")
system_prompt = system_prompt_path.read_text(encoding="utf-8")

agent = project_client.agents.create_agent(
    model=os.environ.get("AZURE_OPENAI_DEPLOYMENT", "gpt-4o"),
    name="Service Agent — KB Assistant",
    instructions=system_prompt,
    toolset=[knowledge_mcp],
)

thread = project_client.agents.threads.create()

print(f"Service Agent created: {agent.id}")
print(f"Thread created: {thread.id}")

In [ ]:
# Cell 5: KB Query — "How do I reset a customer's API key?"

query = "How do I reset a customer's API key?"

project_client.agents.messages.create(
    thread_id=thread.id,
    role="user",
    content=query,
)

run = project_client.agents.runs.create_and_process(
    thread_id=thread.id,
    agent_id=agent.id,
)

print(f"Run status: {run.status}")

In [ ]:
# Cell 6: Display KB article results with citations
from IPython.display import Markdown, display

messages = project_client.agents.messages.list(thread_id=thread.id)

for msg in messages:
    if msg.role == "assistant":
        for content_block in msg.content:
            if hasattr(content_block, "text"):
                display(Markdown(content_block.text.value))
        break

In [ ]:
# Cell 7: Follow-up — "What about SSO configuration?"

followup = "What about SSO configuration?"

project_client.agents.messages.create(
    thread_id=thread.id,
    role="user",
    content=followup,
)

run = project_client.agents.runs.create_and_process(
    thread_id=thread.id,
    agent_id=agent.id,
)

messages = project_client.agents.messages.list(thread_id=thread.id)
for msg in messages:
    if msg.role == "assistant":
        for content_block in msg.content:
            if hasattr(content_block, "text"):
                display(Markdown(content_block.text.value))
        break

In [ ]:
# Cell 8: Cleanup

project_client.agents.delete_agent(agent.id)
print(f"Agent {agent.id} deleted.")

project_client.agents.threads.delete(thread.id)
print(f"Thread {thread.id} deleted.")

print("\nKB assistant session complete.")